In [61]:
import sys; sys.path.insert(0, '../')

import matplotlib.pyplot as plt
import torch
import json
import numpy as np
import torch
import pyqg_subgrid_experiments as pse
from models.cgan_model import CGANModel
from models.mean_var_model import MeanVarModel
from tools.plot_helpers import *
from tools.computational_tools import *
from tools.cnn_tools import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
ds_train = pse.Dataset('/scratch/pp2681/pyqg_NN/mean_var/train.nc')

In [63]:
ds_test = pse.Dataset('/scratch/pp2681/pyqg_NN/mean_var/test.nc')

# Estimation of confidence interval for spectral error

In [64]:
model = torch.load('/scratch/pp2681/pyqg_NN/CGAN_residual_track_Jun07/EXP1/net_state', map_location='cpu')

In [79]:
x_scale = model.x_scale
y_scale = model.y_scale
inputs = model.inputs
targets = model.targets

In [66]:
 # Code from cgan_model.py
def fft_covariance(_y):
    '''
    Computes joint target-target spectral covariance density
    '''
    # Concatenate input-target matrix
    Afft = torch.fft.rfftn(_y, dim=(2,3))
    n = Afft.shape[1]
    C = torch.ones(n,n,*Afft.shape[-2:], device=_y.device)
    C = float('nan') * C
    alpha = 1. / (_y.shape[-2]*_y.shape[-1])
    for i in range(n):
        for j in range(i,n):
            C[i,j,:,:] = torch.mean(alpha*torch.real(Afft[:,i] * torch.conj(Afft[:,j])), dim=0)
    return C

In [67]:
def compute_spectral_distribution(ds, targets, run=slice(0,25)):
    y = dataset_to_array(ds.isel(run=run), targets)
    y = torch.tensor(y_scale.normalize(y))
    return fft_covariance(y).numpy()

In [68]:
def distance_C(C_fake, C_target):
    num = np.mean((C_target-C_fake)**2, axis=(2,3))
    den = np.mean(C_target**2, axis=(2,3))
    return np.nanmean((num / den)**0.5)

In [69]:
def mean_std_distance_C(C):
    d = []
    for i in range(len(C)):
        for j in range(i+1,len(C)):
            d.append(distance_C(C[i], C[j]))
    d = np.array(d)
    print('Mean value: %.4f' % (d.mean()))
    print('1-sigma fold: [%.4f,%.4f]' % (d.mean()-d.std(),d.mean()+d.std()))
    print('5-95 CI: [%.4f,%.4f]' % (np.percentile(d,5), np.percentile(d,95)))
    return

# Data limit: spectral error L2_q

In [70]:
C = []
chunk=25
for run in [slice(0+chunk*j,chunk*(j+1)) for j in range(250//chunk)]:
    print(run)
    C.append(compute_spectral_distribution(ds_train, targets, run=run))

slice(0, 25, None)
slice(25, 50, None)
slice(50, 75, None)
slice(75, 100, None)
slice(100, 125, None)
slice(125, 150, None)
slice(150, 175, None)
slice(175, 200, None)
slice(200, 225, None)
slice(225, 250, None)


In [71]:
mean_std_distance_C(C)

Mean value: 0.0818
1-sigma fold: [0.0781,0.0855]
5-95 CI: [0.0773,0.0899]


# Data limit: spectral error L2_dq

In [72]:
C = []
chunk=25
targets_res = [('q_forcing_advection_res', 0), ('q_forcing_advection_res', 1)]
for run in [slice(0+chunk*j,chunk*(j+1)) for j in range(250//chunk)]:
    print(run)
    C.append(compute_spectral_distribution(ds_train, targets_res, run=run))

slice(0, 25, None)
slice(25, 50, None)
slice(50, 75, None)
slice(75, 100, None)
slice(100, 125, None)
slice(125, 150, None)
slice(150, 175, None)
slice(175, 200, None)
slice(200, 225, None)
slice(225, 250, None)


In [73]:
mean_std_distance_C(C)

Mean value: 0.1048
1-sigma fold: [0.0957,0.1140]
5-95 CI: [0.0899,0.1224]


# Regression: L2_q

In [74]:
C_fake = []
C_target = []
regressor = [('q_forcing_advection_mean', 0), ('q_forcing_advection_mean', 1)]
chunk=25
for run in [slice(0+chunk*j,chunk*(j+1)) for j in range(250//chunk)]:
    C_target.append(compute_spectral_distribution(ds_train, targets, run=run))
    C_fake.append(compute_spectral_distribution(ds_train, regressor, run=run))
    print(run)

slice(0, 25, None)
slice(25, 50, None)
slice(50, 75, None)
slice(75, 100, None)
slice(100, 125, None)
slice(125, 150, None)
slice(150, 175, None)
slice(175, 200, None)
slice(200, 225, None)
slice(225, 250, None)


In [75]:
d=[]
for j in range(len(C_fake)):
    d.append(distance_C(C_fake[j], C_target[j]))
d = np.array(d)
print('------------ metric on the same x -----------')
print('Mean value: %.4f' % (d.mean()))
print('1-sigma fold: [%.4f,%.4f]' % (d.mean()-d.std(),d.mean()+d.std()))
print('5-95 CI: [%.4f,%.4f]' % (np.percentile(d,5), np.percentile(d,95)))

------------ metric on the same x -----------
Mean value: 0.2121
1-sigma fold: [0.2101,0.2142]
5-95 CI: [0.2091,0.2147]


# GZ2021: L2_q

In [81]:
std = ds_train.q_forcing_advection_std
ds_train['q_forcing_advection_res_gen'] = std * np.random.randn(*std.shape)
ds_train['q_forcing_advection_gen'] = ds_train['q_forcing_advection_mean']+ds_train['q_forcing_advection_res_gen']
GZ_targets = [('q_forcing_advection_gen', 0), ('q_forcing_advection_gen', 1)]
GZ_res = [('q_forcing_advection_res_gen', 0), ('q_forcing_advection_res_gen', 1)]

In [82]:
C_fake = []
C_target = []
chunk=25
for run in [slice(0+chunk*j,chunk*(j+1)) for j in range(250//chunk)]:
    C_target.append(compute_spectral_distribution(ds_train, targets, run=run))
    C_fake.append(compute_spectral_distribution(ds_train, GZ_targets, run=run))
    print(run)

slice(0, 25, None)
slice(25, 50, None)
slice(50, 75, None)
slice(75, 100, None)
slice(100, 125, None)
slice(125, 150, None)
slice(150, 175, None)
slice(175, 200, None)
slice(200, 225, None)
slice(225, 250, None)


In [83]:
d=[]
for j in range(len(C_fake)):
    d.append(distance_C(C_fake[j], C_target[j]))
d = np.array(d)
print('------------ metric on the same x -----------')
print('Mean value: %.4f' % (d.mean()))
print('1-sigma fold: [%.4f,%.4f]' % (d.mean()-d.std(),d.mean()+d.std()))
print('5-95 CI: [%.4f,%.4f]' % (np.percentile(d,5), np.percentile(d,95)))

------------ metric on the same x -----------
Mean value: 0.2077
1-sigma fold: [0.2057,0.2098]
5-95 CI: [0.2046,0.2101]


# GZ2021: L2_dq

In [84]:
C_fake = []
C_target = []
chunk=25
for run in [slice(0+chunk*j,chunk*(j+1)) for j in range(250//chunk)]:
    C_target.append(compute_spectral_distribution(ds_train, targets_res, run=run))
    C_fake.append(compute_spectral_distribution(ds_train, GZ_res, run=run))
    print(run)

slice(0, 25, None)
slice(25, 50, None)
slice(50, 75, None)
slice(75, 100, None)
slice(100, 125, None)
slice(125, 150, None)
slice(150, 175, None)
slice(175, 200, None)
slice(200, 225, None)
slice(225, 250, None)


In [85]:
d=[]
for j in range(len(C_fake)):
    d.append(distance_C(C_fake[j], C_target[j]))
d = np.array(d)
print('------------ metric on the same x -----------')
print('Mean value: %.4f' % (d.mean()))
print('1-sigma fold: [%.4f,%.4f]' % (d.mean()-d.std(),d.mean()+d.std()))
print('5-95 CI: [%.4f,%.4f]' % (np.percentile(d,5), np.percentile(d,95)))

------------ metric on the same x -----------
Mean value: 0.9248
1-sigma fold: [0.9241,0.9256]
5-95 CI: [0.9236,0.9257]


# Summary

- Regression model: L2=0.27, L2_q=0.2121, L2_dq=1
- GZ2021: L2=0.27, L2_q=0.2077, L2_dq=0.9248
- Data limit: L2=0, L2_q=0.08, L2_dq=0.1